In [1]:
!pip install transformers

In [2]:
from transformers import  AutoModelForSeq2SeqLM, AutoModelForTokenClassification, AutoModelForSequenceClassification, AutoTokenizer, pipeline, Seq2SeqTrainer, Seq2SeqTrainingArguments, DefaultDataCollator,TrainingArguments, Trainer, get_cosine_schedule_with_warmup

2024-07-25 10:20:08.320727: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 10:20:08.320870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 10:20:08.479205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import pandas as pd
from torch.utils.data import *
from torch.nn import *
from sklearn.metrics import *
import numpy as np
import torch
import os
import re


seed = 221
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# example_input = input()
example_input = "I met Jimmy in Paris last weekend"

# Token Count

In [5]:
def cleaning_text(text):
    text = text.strip()
    text = re.sub("r\s+", " ", text)
    return text
clean_input = cleaning_text(example_input)

token_length = len(clean_input.split())
print(token_length)

7


# Sentiment Classification

In [6]:
# Vietnamese
sc_model_name_vn = "5CD-AI/Vietnamese-Sentiment-visobert"

# English
sc_model_name = "ProsusAI/finbert"
sc_tokenizer = AutoTokenizer.from_pretrained(sc_model_name)
sc_model = AutoModelForSequenceClassification.from_pretrained(sc_model_name)

sc_pipe = pipeline("text-classification", model = sc_model, tokenizer = sc_tokenizer, return_all_scores=True)
sc_output = sc_pipe(clean_input)
sc_output

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'positive', 'score': 0.07297934591770172},
  {'label': 'negative', 'score': 0.024343784898519516},
  {'label': 'neutral', 'score': 0.9026768803596497}]]

# NER - Token Classification

In [7]:
# Vietnamese
ner_model_name_vn = "NlpHUST/ner-vietnamese-electra-base"

# English
ner_model_name = "dslim/bert-base-NER"
ner_tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model_name)

ner_pipe = pipeline("ner", model = ner_model, tokenizer = ner_tokenizer)
ner_output = ner_pipe(clean_input)
print(ner_output)

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'B-PER', 'score': 0.99911255, 'index': 3, 'word': 'Jimmy', 'start': 6, 'end': 11}, {'entity': 'B-LOC', 'score': 0.999676, 'index': 5, 'word': 'Paris', 'start': 15, 'end': 20}]


# Summarization

In [8]:
output_min_length = int(token_length / 2)
output_max_length = token_length


sum_model_name = "facebook/bart-large-cnn"
sum_tokenizer = AutoTokenizer.from_pretrained(sum_model_name)
sum_model = AutoModelForSeq2SeqLM.from_pretrained(sum_model_name)

sum_pipe = pipeline("summarization", model = sum_model, tokenizer = sum_tokenizer, min_length = output_min_length ,max_length = output_max_length)
sum_output = sum_pipe(clean_input)
sum_output

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

[{'summary_text': 'I met Jimmy in'}]

# Emotion 

In [9]:
emo_model_name = "j-hartmann/emotion-english-distilroberta-base"
emo_tokenizer = AutoTokenizer.from_pretrained(emo_model_name)
emo_model = AutoModelForSequenceClassification.from_pretrained(emo_model_name)

emo_pipe = pipeline("text-classification", model = emo_model, tokenizer = emo_tokenizer, return_all_scores=True)
emo_output = emo_pipe(clean_input)
print(emo_output)

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

[[{'label': 'anger', 'score': 0.0014699161984026432}, {'label': 'disgust', 'score': 0.0009604844963178039}, {'label': 'fear', 'score': 0.003682229435071349}, {'label': 'joy', 'score': 0.9256515502929688}, {'label': 'neutral', 'score': 0.018994418904185295}, {'label': 'sadness', 'score': 0.00703013502061367}, {'label': 'surprise', 'score': 0.04221121221780777}]]
